In [1]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

def getName(nome):
    city = dft[get_row(dft, nome).values].index[0]
    return city

In [2]:
def new_route(schedule):
    times = []
    keys = []
    #row = dft[get_row(dft, 'Cremona, sesto 39').values]
    row = dft[get_row(dft, schedule[0]).values]
    key1 = ''
    key2 = ''
    impossibile = []
    
    for i in range(1,len(schedule)):
        key1 = row.index[0]
        try:
            paese=schedule[i]
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    times1 = times[:]
    return steps_times1

In [3]:
#first attempt to local search algorithm
def find_best_position(steps_times, to_insert):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('-')
        row1 = dft[get_row(dft, cities[0].strip()).values]
        #time from city 1 to current (to insert)
        f_c1_to_curr = row1.loc[:,get_column(dft, to_insert).values].values[0][0]
        row2 = dft[get_row(dft, to_insert).values]
        #time from current (to insert) to city 2
        f_curr_to_c2 = row2.loc[:,get_column(dft, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

In [4]:
#returns a pd.Series: keys = city1 - city2, values = times from city1 to city2
def schedule_time(new_schedule):
    times = []
    #row = dft[get_row(dft, 'Cremona, sesto 39').values]
    row = dft[get_row(dft, new_schedule[0]).values]
    keys = []
    key1 = ''
    key2 = ''
    for i in range(1,len(new_schedule)):
        key1 = row.index[0]
        paese=new_schedule[i]

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

In [141]:
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")
compute_ls(fur2, 'San Giovanni in Croce, Feudatari, 25 A', 2)

martedi
mercoledi
giovedi


In [21]:
def timeDistance(start, end):
    start_city_row = dft[get_row(dft, start).values]
    return start_city_row.loc[:,get_column(dft, end).values].values[0][0]

In [22]:
#Data una lista ottenuta da route restituisce la lista dei nomi delle città
def cities_list(serie):
    cities = []
    keys = serie.keys()
    for i in range(0, len(serie)):
        cities = cities + [keys[i].split('-')[0].strip()]
    if len(keys[i].split('-')) > 2: 
        return cities + [keys[i].split('-')[len(keys[i].split('-'))-1].strip()]
    else:
        return cities + [keys[i].split('-')[1].strip()]

In [23]:
pre_schedule = new_route(fur1["lunedi"])
def best_index(lista, schedule, to_insert):
    idx = find_best_position(schedule, to_insert).idxmin()
    a = idx.split("-")[0].strip()    
    try:
        return lista.index(a)+1
    except:
        #problema nome diverso
        if(a == 'Casale Cremasco'):
            a = 'Casale Cremasco-Vidolasco, Roma, 56'
            return lista.index(a)+1
        elif(city == 'gadesco'):
            a = 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
            return lista.index(a)+1

In [40]:
def definitive_local_search(fur):
    #Schedulazione iniziale, e sequenza dei nomi iniziali, una per ciclare e una copia per effettuare pop e insert
    pre_schedule = new_route(fur)
    lista = [getName(x) for x in fur]
    sequence = lista[:]
    #Costo iniziale del percorso greedy
    pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0]) + timeDistance(lista[-1], 'Ostiano, Mazzini, 1')
    new_cost = 0
    local_best_cost = math.inf
    local_best_schedule = []

    #Eseguo finchè ho dei miglioramenti e quindi i nuovi costi sono migliori del migliore al passo prima
    while(True):
        #Prendo ogni città in lista, la inserisco nella posizione migliore, e tra tutte le città prendo quella che da lo
        #spostamento più vantaggioso.
        #Se questo è anche migliore del risultato precedente utilizzo la nuova sequenza come passo del ciclo successivo
        for i in range(len(lista)):
            #Copia della lista su cui lavorare
            sequence = lista[:]
            to_insert = lista[i]
            sequence.remove(to_insert)
            curr_schedule = new_route(sequence)
            index = best_index(sequence, curr_schedule, to_insert)
            sequence.insert(index, to_insert)
            curr_schedule = new_route(sequence)
            new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0]) + timeDistance(sequence[-1], 'Ostiano, Mazzini, 1')
            if(new_cost<local_best_cost):
                local_best_cost = new_cost
                local_best_schedule = curr_schedule[:]
        if(local_best_cost<pre_cost):
            pre_cost = local_best_cost
            pre_schedule = local_best_schedule[:]
            lista = cities_list(pre_schedule)
        else:
            break
    return cities_list(pre_schedule), pre_cost

In [42]:
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
for day in week:
    sch, cst = definitive_local_search(fur1[day].dropna())
    print(*sch, sep="\n")
    print(cst)

Sesto ed Uniti, Roma, 4
Grumello Cremonese ed Uniti, Roma, 2
Pizzighettone, Giuseppe Garibaldi, 18
Soresina, Giacomo Matteotti, 6
Trigolo, Roma, 26
Castelleone, Roma, 67
Montodine,  XXV Aprile, 10
Ripalta Cremasca, Roma, 5
Ricengo, Guglielmo Marconi
Capralba, Piave, 2
Sergnano, IV Novembre,  10
Casale Cremasco
Camisano, Trieste, 57
Soncino, Dante Alighieri, 23
183.97
Cremona, Ugolani Dati, 4
Persico Dosimo, Albert ,7
Ostiano, Mazzini, 1
San Giovanni in Croce, Feudatari, 25 A
Casalmaggiore, Guglielmo Marconi, 8
Scandolara Ravara, Italia
Casalpusterlengo, Pusterla 7
Vailate, Dante Alighieri, 7
Crema, Civerchi, 9
253.79
Castelverde, Ubaldo Ferrari, 15
Annicco, Giuseppe Garibaldi, 6
Castelleone, Roma, 67
Madignano, Portici, 4
Bagnolo Cremasco, Roma, 3
Palazzo Pignano, Marco Polo, 31
Pandino, Castello, 15
Rivolta d'Adda, Stefano Renzi, 5
Spino d'Adda, Circonvallazione, 13
Romanengo, Guaiarini, 14
164.64
Cremona, Sesto 39
Cremona, Ugolani Dati, 4
Persico Dosimo, Albert ,7
Grontardo, Roma, 16

In [32]:
pre_cost

183.97

In [17]:
calcolato_da_max = pd.read_csv("../Dati/turni_f/ls1_shifts.csv", delimiter=";")

In [48]:
sum(new_route(calcolato_da_max["venerdi"].dropna()).values)

180.0

In [29]:
pre_schedule

Sesto ed Uniti, Roma, 4 - Grumello Cremonese ed Uniti, Roma, 2            6.0
Grumello Cremonese ed Uniti, Roma, 2 - Soresina, Giacomo Matteotti, 6    12.0
Soresina, Giacomo Matteotti, 6 - Trigolo, Roma, 26                        9.0
Trigolo, Roma, 26 - Castelleone, Roma, 67                                11.0
Castelleone, Roma, 67 - Montodine,  XXV Aprile, 10                        7.0
Montodine,  XXV Aprile, 10 - Ripalta Cremasca, Roma, 5                    5.0
Ripalta Cremasca, Roma, 5 - Ricengo, Guglielmo Marconi                   12.0
Ricengo, Guglielmo Marconi - Sergnano, IV Novembre,  10                   7.0
Sergnano, IV Novembre,  10 - Casale Cremasco-Vidolasco, Roma, 56          4.0
Casale Cremasco-Vidolasco, Roma, 56 - Camisano, Trieste, 57               6.0
Camisano, Trieste, 57 - Capralba, Piave, 2                               15.0
Capralba, Piave, 2 - Soncino, Dante Alighieri, 23                        21.0
Soncino, Dante Alighieri, 23 - Pizzighettone, Giuseppe Garibaldi